# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
client_key = 'XBzs64lBfmirUeVb12vQ39Rvz' #write your API key
client_secret = '0OA5V1a2tahS7o9Q98HtesTlq3JjUb9e4OHunsMwJsAsLYsWiz'  #write your API secret key 

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAACDjegEAAAAAtcxjMpKJ1wAgtaa16B6%2BggItGAc%3DPF5TMVaueFg8IrRcYyfNNHf8ZF1vLqtfnnKT9Td5qQUPlpb6Ha'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
climate_tweets = response.json()


In [10]:
len(climate_tweets['data'])

100

In [11]:
tweets +=climate_tweets['data']


In [12]:

ids = [tw['id'] for tw in climate_tweets['data']]
    

In [13]:
min_id = min(ids)

In [14]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [15]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'until_id' : min_id,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
                 }

for  i in range(max_tweets//tw_limit ):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['data']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

searching tweets with id  < 1549091170755674114
0 100


In [16]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 200


# Let's print some tweets

In [17]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [18]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '21513363',
  'conversation_id': '1549061432804663296',
  'created_at': '2022-07-18T17:58:40.000Z',
  'entities': {'mentions': [{'end': 13,
                             'id': '427018473',
                             'start': 0,
                             'username': 'lizzienugent'}]},
  'geo': {'place_id': '3797791ff9c0e4c6'},
  'id': '1549091298463961088',
  'in_reply_to_user_id': '427018473',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 0},
  'referenced_tweets': [{'id': '1549088828853559298', 'type': 'replied_to'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'text': '@lizzienugent Whatever the essence of your personality is, your '
          'inner climate…it is only enunciated with money/toys. Doesn’t '
          'necessarily change you, I don’t believe. You’re either happy or '
          'you’re not.'},
 {'author_id

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [19]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

@lizzienugent Whatever the essence of your personality is, your inner climate…it is only enunciated with money/toys. Doesn’t necessarily change you, I don’t believe. You’re either happy or you’re not.

RT @lifesupremacist: A gathering of white people on colonized land watching a wave approaching that has the power to take them out and not…

RT @all_americans20: Well, Well ,Well , PRAISE GOES OUT TO ** BERNIE SANDERS for Calling Out j.manch , for the ONE PERSON - / DEMOCCRAT : F…

Our kids deserve a future. Join me in demanding that our elected leaders take action to tackle climate change and protect kids from pollution: https://t.co/Jnfh0HkEfV



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [20]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['entities', 'lang', 'reply_settings', 'text', 'public_metrics', 'id', 'conversation_id', 'referenced_tweets', 'source', 'author_id', 'created_at'])
11


In [21]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [22]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [23]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'entities': {'mentions': [{'start': 0,
     'end': 13,
     'username': 'lizzienugent',
     'id': '427018473'}]},
  'lang': 'en',
  'reply_settings': 'everyone',
  'text': '@lizzienugent Whatever the essence of your personality is, your inner climate…it is only enunciated with money/toys. Doesn’t necessarily change you, I don’t believe. You’re either happy or you’re not.',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'in_reply_to_user_id': '427018473',
  'id': '1549091298463961088',
  'conversation_id': '1549061432804663296',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1549088828853559298'}],
  'geo': {'place_id': '3797791ff9c0e4c6'},
  'source': 'Twitter for iPhone',
  'created_at': '2022-07-18T17:58:40.000Z',
  'user_id': '21513363',
  'referenced_tweets_ids': '1549088828853559298',
  'retweet_count': 0},
 {'entities': {'mentions': [{'start': 3,
     'end': 19,
     'username': 'lifesupremacist',
     'id': '11

In [24]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [25]:
climate_feed_df

,entities,lang,reply_settings,text,public_metrics,in_reply_to_user_id,id,conversation_id,referenced_tweets,geo,source,created_at,user_id,referenced_tweets_ids,retweet_count
0,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",en,everyone,@lizzienugent Whatever the essence of your per...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",427018473,1549091298463961088,1549061432804663296,"[{'type': 'replied_to', 'id': '154908882885355...",{'place_id': '3797791ff9c0e4c6'},Twitter for iPhone,2022-07-18T17:58:40.000Z,21513363,1549088828853559298,0
1,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",en,everyone,RT @lifesupremacist: A gathering of white peop...,"{'retweet_count': 943, 'reply_count': 0, 'like...",NaN,1549091295783636992,1549091295783636992,"[{'type': 'retweeted', 'id': '1549064032522383...",NaN,Twitter Web App,2022-07-18T17:58:40.000Z,369820584,1549064032522383360,943
2,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",en,everyone,"RT @all_americans20: Well, Well ,Well , PRAISE...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,1549091293728591873,1549091293728591873,"[{'type': 'retweeted', 'id': '1549091231510286...",NaN,Twitter for Android,2022-07-18T17:58:39.000Z,1254107497310433286,1549091231510286336,1
3,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",en,everyone,RT @GreenGreenwich: France braces for record t...,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",NaN,1549091292621283329,1549091292621283329,"[{'type': 'retweeted', 'id': '1548918559111147...",NaN,Twitter for iPhone,2022-07-18T17:58:39.000Z,1176236156200972289,1548918559111147520,3
4,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,everyone,RT @GeorgeMonbiot: I argue that mainstream env...,"{'retweet_count': 581, 'reply_count': 0, 'like...",NaN,1549091292356935681,1549091292356935681,"[{'type': 'retweeted', 'id': '1549048350036885...",NaN,Twitter Web App,2022-07-18T17:58:39.000Z,1287182387768692737,1549048350036885504,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"{'mentions': [{'start': 3, 'end': 7, 'username...",en,everyone,RT @LBC: Andrew Marr has hit out at 'sleazy an...,"{'retweet_count': 7, 'reply_count': 0, 'like_c...",NaN,1549091036487630849,1549091036487630849,"[{'type': 'retweeted', 'id': '1549088143407624...",NaN,Twitter for iPhone,2022-07-18T17:57:38.000Z,1522542908708114433,1549088143407624192,7
184,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,everyone,RT @David_Moscrop: Millennials are so greedy. ...,"{'retweet_count': 131, 'reply_count': 0, 'like...",NaN,1549091034466058245,1549091034466058245,"[{'type': 'retweeted', 'id': '1549028179993583...",NaN,Twitter Web App,2022-07-18T17:57:37.000Z,33625956,1549028179993583619,131
185,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",en,everyone,RT @JeremyClarkson: I’ve spent my life working...,"{'retweet_count': 3350, 'reply_count': 0, 'lik...",NaN,1549091033262297088,1549091033262297088,"[{'type': 'retweeted', 'id': '1549015974556336...",NaN,Twitter for iPhone,2022-07-18T17:57:37.000Z,2644615089,1549015974556336128,3350
186,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",en,everyone,RT @TheGreatRESlST: 🚨🚨 WEF has revealed plans ...,"{'retweet_count': 310, 'reply_count': 0, 'like...",NaN,1549091032792346624,1549091032792346624,"[{'type': 'retweeted', 'id': '1548767777166606...",NaN,Twitter for iPhone,2022-07-18T17:57:37.000Z,419970338,1548767777166606337,310


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [26]:
# write code here datatype listing
climate_feed_df.dtypes
# the created_at field should be datetime 

entities                 object
lang                     object
reply_settings           object
text                     object
public_metrics           object
in_reply_to_user_id      object
id                       object
conversation_id          object
referenced_tweets        object
geo                      object
source                   object
created_at               object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
dtype: object

# Write columns with wrong datatype and mention correct data type too.

the created_at field should be datetime; entities, public_metrics and referenced_tweets should be lists


# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [27]:
def tweet_user_generator(tweets):
# write code hered
    return tweets['includes']['users']

In [28]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [29]:
user_df

,verified,username,name,profile_image_url,id,description,url,location,protected,public_metrics,created_at,entities,pinned_tweet_id
0,False,drewrhiw,Andy Pike,https://pbs.twimg.com/profile_images/148738693...,507861681,"Guy Fawkes was right Life is short, Break the ...",,"Cardiff, Wales",False,"{'followers_count': 577, 'following_count': 12...",2012-02-28T20:15:16.000Z,NaN,NaN
1,False,BCisvulnerable,@🇨🇦До біса Путіна” 4th Vax,https://pbs.twimg.com/profile_images/151790045...,2812369343,#StandWithUkraine Social Democrat\n#abortionr...,,BC,False,"{'followers_count': 1348, 'following_count': 1...",2014-10-07T02:28:09.000Z,"{'description': {'hashtags': [{'start': 0, 'en...",1544123583093284864
2,False,JeanneGary13,Jeanne Gary,https://pbs.twimg.com/profile_images/154008540...,1540085055435022336,"Grandmother, soap maker, crafter, dancer, liberal",,NaN,False,"{'followers_count': 13, 'following_count': 139...",2022-06-23T21:31:33.000Z,NaN,NaN
3,False,TammyCo18192360,Tammy Cortez (Cortes),https://pbs.twimg.com/profile_images/127948383...,1164657325942157315,,,NaN,False,"{'followers_count': 84, 'following_count': 351...",2019-08-22T21:55:31.000Z,NaN,NaN
4,False,bettedavisthizz,Margo Chatting,https://pbs.twimg.com/profile_images/150487348...,1203350541725773829,All my goodness has turned to badness.,https://t.co/jrekj7Si4W,"Small Town, O",False,"{'followers_count': 2906, 'following_count': 2...",2019-12-07T16:28:37.000Z,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,JohnWil87488469,John Wilson,https://abs.twimg.com/sticky/default_profile_i...,1522542908708114433,,,NaN,False,"{'followers_count': 4, 'following_count': 61, ...",2022-05-06T11:45:22.000Z,NaN,NaN
96,False,Jessica_Nudo,Jessica Nudo ✨,https://pbs.twimg.com/profile_images/140005128...,33625956,"High femme🏳️‍🌈🥦 #OuidPR & #finance Publicist, ...",https://t.co/XtustjPJNu,"Toronto, Ontario",False,"{'followers_count': 1294, 'following_count': 1...",2009-04-20T19:16:27.000Z,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",1246154182534615041
97,False,IanIanandmaria,Ian,https://pbs.twimg.com/profile_images/102468651...,2644615089,"just having a laugh , lifes to short not to la...",,"Stockport, England",False,"{'followers_count': 425, 'following_count': 11...",2014-06-26T18:36:58.000Z,NaN,NaN
98,False,tubelessready,Lawny Moore,https://pbs.twimg.com/profile_images/149183195...,419970338,"Mountain bike racer and rider, not interested ...",,"British Columbia, Canada",False,"{'followers_count': 203, 'following_count': 95...",2011-11-24T01:15:55.000Z,NaN,NaN


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [30]:
# write code here for datatype listing
user_df.dtypes

verified               bool
username             object
name                 object
profile_image_url    object
id                   object
description          object
url                  object
location             object
protected              bool
public_metrics       object
created_at           object
entities             object
pinned_tweet_id      object
dtype: object

# write columns with wrong datatype and mention correct data type too.

created at should be datetime, public_metrics and entities should be a list

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [31]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [32]:
retweeted_status_df.head()

,id,retweet_count
0,1549091298463961088,0
1,1549091295783636992,943
2,1549091293728591873,1
3,1549091292621283329,3
4,1549091292356935681,581


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [33]:
# Write code here
print(len(retweeted_status_df[retweeted_status_df['retweet_count']==0]))
retweeted_status_df[retweeted_status_df['retweet_count']==0]

34


,id,retweet_count
0,1549091298463961088,0
9,1549091287885910016,0
16,1549091271867875328,0
23,1549091252104318976,0
25,1549091251454189569,0
26,1549091249822568448,0
48,1549091219371876355,0
51,1549091215899242496,0
53,1549091214049296390,0
55,1549091212015112192,0
